In [1]:
import pandas as pd
import numpy as np
import copy

import math
import scipy.stats as stats

import datetime
from dateutil.relativedelta import relativedelta

import matplotlib.pyplot as plt

# !pip install pypsa
import pypsa
# !pip install cartopy
import cartopy

import warnings
warnings.filterwarnings('ignore')

C:\Users\ywu\AppData\Local\anaconda3\lib\site-packages\paramiko\transport.py:219: CryptographyDeprecationWarning: Blowfish has been deprecated
  "class": algorithms.Blowfish,


In [2]:
# Import sample network
network_sample = pypsa.examples.ac_dc_meshed(from_master=True)

n = network_sample
n.plot(
    line_colors=n.lines.bus0.map(n.buses.carrier).map(lambda ct: "r" if ct == "DC" else "b"),
    title="Mixed AC (blue) - DC (red) network - DC (cyan)",
    color_geomap=True,
    jitter=0.3,
)
plt.tight_layout()

INFO:pypsa.io:Imported network ac-dc-meshed.nc has buses, carriers, generators, global_constraints, lines, links, loads


In [3]:
# Create a network with only AC lines

network = pypsa.Network()

network.buses = copy.deepcopy(network_sample.buses[network_sample.buses['carrier'] == 'AC'])

network.lines = copy.deepcopy(network_sample.lines.iloc[[0, 1, 5, 6],])
network.lines.index = ['London-Manchester', 'Manchester-Norwich', 'Norwich-London', 'Bremen-Frankfurt']
network.add('Line', 'Norway-Bremen', bus0='Norway', bus1='Bremen', \
            capital_cost=0.14, s_nom=40000, s_nom_extendable=True, x=0.5, r=0)
network.add('Line', 'Norway-Norwich', bus0='Norway', bus1='Norwich', \
            capital_cost=0.15, s_nom=40000, s_nom_extendable=True, x=0.55, r=0)
network.add('Line', 'Norway-Manchester', bus0='Norway', bus1='Manchester', \
            capital_cost=0.16, s_nom=40000, s_nom_extendable=True, x=0.6, r=0)

network.generators = copy.deepcopy(network_sample.generators)

n = network
n.plot(
    line_colors=n.lines.bus0.map(n.buses.carrier).map(lambda ct: "r" if ct == "DC" else "b"),
    title="Mixed AC (blue) - DC (red) network - DC (cyan)",
    color_geomap=True,
    jitter=0.3,
)
plt.tight_layout()

In [4]:
# Buses in the network

# name: Unique name
# v_nom: Nominal voltage
# x: Position (e.g. longitude)
# y: Position (e.g. latitude)
# carrier: Energy carrier (AC or DC for electrical buses, or heat or gas)

# How to add a bus
# network.add('Bus', 'Name', v_nom=...)

network.buses

v_nom carrier      x      y type  unit  v_mag_pu_set  \
Bus                                                                
London      380.0      AC  -0.13  51.50       None           1.0   
Norwich     380.0      AC   1.30  52.60       None           1.0   
Manchester  380.0      AC  -2.20  53.47       None           1.0   
Bremen      380.0      AC   8.80  53.08       None           1.0   
Frankfurt   380.0      AC   8.70  50.12       None           1.0   
Norway      380.0      AC  10.75  60.00       None           1.0   

            v_mag_pu_min  v_mag_pu_max control sub_network  
Bus                                                         
London               0.0           inf      PQ              
Norwich              0.0           inf      PQ              
Manchester           0.0           inf      PQ              
Bremen               0.0           inf      PQ              
Frankfurt            0.0           inf      PQ              
Norway               0.0           inf      PQ

In [5]:
# Lines in the network

# name: Unique name
# bus0 and bus1: Buses to which line is attached
# x: Series reactance, must be non-zero for AC branch in linear power flow
# r: Series resistance, must be non-zero for DC branch in linear power flow
# s_nom: Limit of apparent power which can pass through line
# capital_cost: Capital cost of extending s_nom by 1 MVA

# How to add a line
# network.add('Line', 'Name', bus0=...)

network.lines['s_nom'] = 8000
network.lines['s_nom_extendable'] = False
network.lines

bus0        bus1  capital_cost  s_nom  \
Line                                                              
London-Manchester       London  Manchester      0.136716   8000   
Manchester-Norwich  Manchester     Norwich      0.133492   8000   
Norwich-London         Norwich      London      0.021852   8000   
Bremen-Frankfurt        Bremen   Frankfurt      0.200000   8000   
Norway-Bremen           Norway      Bremen      0.140000   8000   
Norway-Norwich          Norway     Norwich      0.150000   8000   
Norway-Manchester       Norway  Manchester      0.160000   8000   

                    s_nom_extendable         x    r type    g    b  ...  \
Line                                                                ...   
London-Manchester              False  0.796878  0.0       0.0  0.0  ...   
Manchester-Norwich             False  0.391560  0.0       0.0  0.0  ...   
Norwich-London                 False  0.238800  0.0       0.0  0.0  ...   
Bremen-Frankfurt               False  0.400000  0.0       0.0  0.0  ...   
Norway-Bremen                  False  0.500000  0.0       0.0  0.0  ...   
Norway-Norwich                 False  0.550000  0.0       0.0  0.0  ...   
Norway-Manchester              False  0.600000  0.0       0.0  0.0  ...   

                    v_ang_min  v_ang_max  sub_network  x_pu  r_pu  g_pu b_pu  \
Line                                                                           
London-Manchester        -inf        inf                0.0   0.0   0.0  0.0   
Manchester-Norwich       -inf        inf                0.0   0.0   0.0  0.0   
Norwich-London           -inf        inf                0.0   0.0   0.0  0.0   
Bremen-Frankfurt         -inf        inf                0.0   0.0   0.0  0.0   
Norway-Bremen            -inf        inf                0.0   0.0   0.0  0.0   
Norway-Norwich           -inf        inf                0.0   0.0   0.0  0.0   
Norway-Manchester        -inf        inf                0.0   0.0   0.0  0.0   

                    x_pu_eff  r_pu_eff  s_nom_opt  
Line                                               
London-Manchester        0.0       0.0        0.0  
Manchester-Norwich       0.0       0.0        0.0  
Norwich-London           0.0       0.0        0.0  
Bremen-Frankfurt         0.0       0.0        0.0  
Norway-Bremen            0.0       0.0        0.0  
Norway-Norwich           0.0       0.0        0.0  
Norway-Manchester        0.0       0.0        0.0  

[7 rows x 29 columns]

In [6]:
# Generators in the network

# name: Unique name
# bus: Bus to which generator is attached
# p_nom: Nominal power for limits in optimization
# carrier: Prime mover energy carrier (e.g. coal, gas, wind, solar)
# marginal_cost: Marginal cost of production of 1 MWh
# capital_cost: Capital cost of extending p_nom by 1 MW
# efficiency: Ratio between primary energy and electrical energy

# How to add a generator
# network.add('Generator', 'Name', bus=...)

network.generators

bus  capital_cost  efficiency  marginal_cost    p_nom  \
Generator                                                                       
Manchester Wind  Manchester   2793.651603    1.000000       0.110000     80.0   
Manchester Gas   Manchester    196.615168    0.350026       4.532368  50000.0   
Norway Wind          Norway   2184.374796    1.000000       0.090000    100.0   
Norway Gas           Norway    158.251250    0.356836       5.892845  20000.0   
Frankfurt Wind    Frankfurt   2129.456122    1.000000       0.100000    110.0   
Frankfurt Gas     Frankfurt    102.676953    0.351666       4.086322  80000.0   

                 p_nom_extendable  p_nom_min carrier control type  ...  \
Generator                                                          ...   
Manchester Wind              True      100.0    wind      PQ       ...   
Manchester Gas               True        0.0     gas      PQ       ...   
Norway Wind                  True      100.0    wind      PQ       ...   
Norway Gas                   True        0.0     gas      PQ       ...   
Frankfurt Wind               True      100.0    wind      PQ       ...   
Frankfurt Gas                True        0.0     gas      PQ       ...   

                 shut_down_cost  min_up_time  min_down_time  up_time_before  \
Generator                                                                     
Manchester Wind             0.0            0              0               1   
Manchester Gas              0.0            0              0               1   
Norway Wind                 0.0            0              0               1   
Norway Gas                  0.0            0              0               1   
Frankfurt Wind              0.0            0              0               1   
Frankfurt Gas               0.0            0              0               1   

                 down_time_before  ramp_limit_up  ramp_limit_down  \
Generator                                                           
Manchester Wind                 0            NaN              NaN   
Manchester Gas                  0            NaN              NaN   
Norway Wind                     0            NaN              NaN   
Norway Gas                      0            NaN              NaN   
Frankfurt Wind                  0            NaN              NaN   
Frankfurt Gas                   0            NaN              NaN   

                 ramp_limit_start_up  ramp_limit_shut_down  p_nom_opt  
Generator                                                              
Manchester Wind                  1.0                   1.0        0.0  
Manchester Gas                   1.0                   1.0        0.0  
Norway Wind                      1.0                   1.0        0.0  
Norway Gas                       1.0                   1.0        0.0  
Frankfurt Wind                   1.0                   1.0        0.0  
Frankfurt Gas                    1.0                   1.0        0.0  

[6 rows x 30 columns]

In [7]:
# Loads in the network

# name, bus, carrier, p_set, q_set seems to be necessary for generators
# name: Unique name
# bus: Bus to which load is attached
# carrier: Energy carrier (AC or DC for electrical buses, or heat or gas)
# p_set: Active power consumption
# q_set: Reactive power consumption

# How to add a load
# network.add('Load', 'Name', bus=...)

network.loads = pd.DataFrame()
network.add('Load', 'Load London', bus='London', p_set=10000)
network.add('Load', 'Load Norwich', bus='Norwich', p_set=10000)
network.add('Load', 'Load Manchester', bus='Manchester', p_set=10000)
network.add('Load', 'Load Bremen', bus='Bremen', p_set=10000)
network.add('Load', 'Load Frankfurt', bus='Frankfurt', p_set=10000)
network.add('Load', 'Load Norway', bus='Norway', p_set=10000)
network.loads

attribute               bus carrier type    p_set  q_set  sign
Load                                                          
Load London          London               10000.0    0.0  -1.0
Load Norwich        Norwich               10000.0    0.0  -1.0
Load Manchester  Manchester               10000.0    0.0  -1.0
Load Bremen          Bremen               10000.0    0.0  -1.0
Load Frankfurt    Frankfurt               10000.0    0.0  -1.0
Load Norway          Norway               10000.0    0.0  -1.0

In [8]:
# Newton-Raphson Power Flow Model

network.pf()

INFO:pypsa.pf:Performing non-linear load-flow on AC sub-network SubNetwork 0 for snapshots Index(['now'], dtype='object', name='snapshot')
INFO:pypsa.pf:Newton-Raphson solved in 4 iterations with error of 0.000000 in 0.019392 seconds


{'n_iter': SubNetwork  0
 snapshot     
 now         4,
 'error': SubNetwork             0
 snapshot                
 now         3.285550e-10,
 'converged': SubNetwork     0
 snapshot        
 now         True}

In [9]:
# Active power flow on the lines

network.lines_t.p0

# Each bus has 10000 load

London-Manchester  Manchester-Norwich  Norwich-London  \
snapshot                                                          
now            -9649.095059        19412.620981      350.904941   

          Bremen-Frankfurt  Norway-Bremen  Norway-Norwich  Norway-Manchester  
snapshot                                                                      
now                10000.0        20000.0    -9061.716041      -20938.283959

In [10]:
# All power comes from Manchester Wind

network.buses

v_nom carrier      x      y type  unit  v_mag_pu_set  \
Bus                                                                
London      380.0      AC  -0.13  51.50       None           1.0   
Norwich     380.0      AC   1.30  52.60       None           1.0   
Manchester  380.0      AC  -2.20  53.47       None           1.0   
Bremen      380.0      AC   8.80  53.08       None           1.0   
Frankfurt   380.0      AC   8.70  50.12       None           1.0   
Norway      380.0      AC  10.75  60.00       None           1.0   

            v_mag_pu_min  v_mag_pu_max control sub_network        generator  
Bus                                                                          
London               0.0           inf      PQ           0              NaN  
Norwich              0.0           inf      PQ           0              NaN  
Manchester           0.0           inf   Slack           0  Manchester Wind  
Bremen               0.0           inf      PQ           0              NaN  
Frankfurt            0.0           inf      PQ           0              NaN  
Norway               0.0           inf      PQ           0              NaN

In [11]:
# Optimal Power Flow Model

network.optimize()

Index(['London-Manchester', 'Manchester-Norwich', 'Norwich-London',
       'Bremen-Frankfurt', 'Norway-Bremen', 'Norway-Norwich',
       'Norway-Manchester'],
      dtype='object', name='Line')
Index(['London-Manchester', 'Manchester-Norwich', 'Norwich-London',
       'Bremen-Frankfurt', 'Norway-Bremen', 'Norway-Norwich',
       'Norway-Manchester'],
      dtype='object', name='Line')
INFO:linopy.model: Solve linear problem using Glpk solver
INFO:linopy.io: Writing time: 0.08s
INFO:linopy.constants: Optimization successful: 
Status: ok
Termination condition: optimal
Solution: 20 primals, 40 duals
Objective: -1.17e+07
Solver model: not available
Solver message: optimal



GLPSOL: GLPK LP/MIP Solver, v4.65
Parameter(s) specified in the command line:
 --lp C:\Users\ywu\AppData\Local\Temp\linopy-problem-vmzab2je.lp --output
 C:\Users\ywu\AppData\Local\Temp\linopy-solve-biylcmfq.sol
Reading problem data from 'C:\Users\ywu\AppData\Local\Temp\linopy-problem-vmzab2je.lp'...
40 rows, 20 columns, 64 non-zeros
277 lines were read
GLPK Simplex Optimizer, v4.65
40 rows, 20 columns, 64 non-zeros
Preprocessing...
11 rows, 13 columns, 29 non-zeros
Scaling...
 A: min|aij| =  1.654e-01  max|aij| =  1.000e+00  ratio =  6.047e+00
Problem data seem to be well scaled
Constructing initial basis...
Size of triangular part is 10
      0: obj =  -1.215188717e+07 inf =   3.181e+04 (4)
      4: obj =  -1.164697701e+07 inf =   0.000e+00 (0)
*     7: obj =  -1.169415250e+07 inf =   0.000e+00 (0)
OPTIMAL LP SOLUTION FOUND
Time used:   0.0 secs
Memory used: 0.1 Mb (56584 bytes)
Writing basic solution to 'C:\Users\ywu\AppData\Local\Temp\linopy-solve-biylcmfq.sol'...


('ok', 'optimal')

In [12]:
# Active power flow on the lines

network.lines_t.p0

# Each bus has 10000 load; lines are restricted to their limits

London-Manchester  Manchester-Norwich  Norwich-London  \
snapshot                                                          
now                -4965.34             7034.66         5034.66   

          Bremen-Frankfurt  Norway-Bremen  Norway-Norwich  Norway-Manchester  
snapshot                                                                      
now                -8000.0         2000.0          8000.0            2742.51

In [13]:
# Optimised nominal power for each generator

network.generators['p_nom_opt']

Generator
Manchester Wind      100.0
Manchester Gas     19157.5
Norway Wind          100.0
Norway Gas         22642.5
Frankfurt Wind       100.0
Frankfurt Gas      17900.0
Name: p_nom_opt, dtype: float64